In [ ]:
#]add GBIF#mkb/dev

In [ ]:
using GBIF

In [ ]:
tax = "Malvaceae"
taxon_rank = :FAMILY
phylum = "plants"
life_form = "terrestrial"

In [ ]:
gbtax = GBIF.taxon(tax, rank = taxon_rank)

In [ ]:
occ = GBIF.occurrences(gbtax)

In [ ]:
occ = GBIF.occurrences(gbtax, Dict{String, Any}("country" => "BR"))

In [ ]:
qualitycontrol!(occ, filters = [have_both_coordinates]);

In [ ]:
gbtax2 = GBIF.taxon.("Pseudobombax", rank = :GENUS)
occ2 = GBIF.occurrences(gbtax2, Dict{String, Any}("limit" => 300, "geometry" => "POLYGON((-34.7 32.8, -117.2 32.8, -117.2 -55.8, -34.7 -55.8, -34.7 32.8))"))

In [ ]:
complete!(occ2);

In [ ]:
using Query, Dates, DataFrames, StatPlots

In [ ]:
dats = occ2 |> 
@filter(ismissing(_.basisOfRecord) || _.basisOfRecord ∈ (:HUMAN_OBSERVATION, :OBSERVATION, :PRESERVED_SPECIMEN)) |>
@filter(ismissing(_.uncertainty) || _.uncertainty/1000 <= 100) |>
@filter(ismissing(_.date) || year(_.date) > 1945) |>
@filter(have_both_coordinates(_)) |>
DataFrame
first(dats, 5)

In [ ]:
@df dats scatter(:longitude, :latitude, 
    group = :basisOfRecord, 
    aspect_ratio = 1, 
    legend = :bottomleft, 
    ms = 3, msc = :white)

In [ ]:
using SpatialEcology

In [ ]:
cord = hcat(dats[:longitude], dats[:latitude])
cord = round.((cord .+ 0.5) ./ 3) .* 3 .- 0.5
sites = vec(mapslices(x->"$(x[1])_$(x[2])", cord, dims = 2))

In [ ]:
cm = ComMatrix(species = dats[:scientific], sites = sites)

In [ ]:
pbom = Assemblage(cm, unique(cord, dims = 1))

In [ ]:
plot(pbom)

In [ ]:
using Proj4

In [ ]:
latlon = Projection("+proj=longlat +datum=WGS84 +no_defs")
behr = Projection("+proj=cea +lon_0=0 +lat_ts=30 +datum=WGS84 +units=m +no_defs")
newcd = transform(latlon, behr, hcat(dats[:longitude], dats[:latitude]))


In [ ]:
scatter(newcd[:,1], newcd[:,2], aspectratio=1)

In [ ]:
using Shapefile

In [ ]:
shp = open("../Data/World_coast/World_coast.shp", "r") do io
    read(io, Shapefile.Handle)
end

In [ ]:
plot(shp.shapes, xlim = extrema(dats[:longitude]), ylim = extrema(dats[:latitude]), c = :lightgrey)
plot!(pbom)

Don't run the code below!
If we want to transform the shapefile too, we can use the GDAL library, in Julia often used with ArchGDAL. This shows how to do the transformation, but it appears to leak memory. 

In [ ]:
using ArchGDAL
const AG = ArchGDAL

In [ ]:
shapes = AG.registerdrivers() do
    AG.read("../Data/World_coast/World_coast.shp") do dataset
        [AG.getgeom(feature) for feature in AG.getlayer(dataset, 0)]
    end
end

In [ ]:
AGbehr = AG.importPROJ4("+proj=cea +lon_0=0 +lat_ts=30 +datum=WGS84 +units=m +no_defs")
AGlatlon = AG.importPROJ4("+proj=longlat +datum=WGS84 +no_defs")
AG.createcoordtrans(AGlatlon, AGbehr) do coordtrans
    AG.transform!.(shapes, Ref(coordtrans))
end

In [ ]:
plot(shapes)